In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import prondict
from lab2_proto import *
from lab2_tools import *
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

data = np.load('lab2_data.npz', allow_pickle=True)['data']
example = np.load('lab2_example.npz', allow_pickle=True)['example'].item()
all_spkr_phone_HMMs = np.load('lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()
one_spkr_phone_HMMs = np.load('lab2_models_onespkr.npz', allow_pickle=True)['phoneHMMs'].item()

# Print keys for each .npz file
print("Keys in 'lab2_data.npz':", list(data[1].keys()))
print("Keys in 'lab2_example.npz':", list(example.keys()))
print("Keys in 'lab2_models_all.npz':", list(all_spkr_phone_HMMs.keys()))
print("Keys in 'lab2_models_onespkr.npz':", list(one_spkr_phone_HMMs.keys()))

# Concat
isolated = {}
for digit in prondict.keys():
  isolated[digit] = ["sil"] + prondict[digit] + ["sil"]

print(isolated)

all_spkr_word_HMMs, one_spkr_word_HMMs = {}, {}
all_spkr_word_HMMs["o"] = concatHMMs(all_spkr_phone_HMMs, isolated["o"])
one_spkr_word_HMMs["o"] = concatHMMs(all_spkr_phone_HMMs, isolated["o"])

# Gaussian Plot
all_obsloglik = log_multivariate_normal_density_diag(example["lmfcc"], all_spkr_word_HMMs["o"]["means"], all_spkr_word_HMMs["o"]["covars"])
one_obsloglik = log_multivariate_normal_density_diag(example["lmfcc"], one_spkr_word_HMMs["o"]["means"], one_spkr_word_HMMs["o"]["covars"])

fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(all_obsloglik.T, cmap='viridis')
ax.set_title('Our obslogik')
plt.show()

fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(one_obsloglik.T, cmap='viridis')
ax.set_title('Our obslogik')
plt.show()

# Forward
logalpha = forward(example["obsloglik"], np.log(one_spkr_word_HMMs["o"]["startprob"]), np.log(one_spkr_word_HMMs["o"]["transmat"]))
fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(logalpha.T, cmap='viridis')
ax.set_title('Our logalpha')
plt.show()

fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(example["logalpha"].T, cmap='viridis')
ax.set_title('Correct logalpha')
plt.show()

# Backward
logbeta = backward(one_obsloglik, np.log(one_spkr_word_HMMs['o']['startprob']), np.log(one_spkr_word_HMMs['o']['transmat']))
fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(logbeta.T, cmap='viridis')
ax.set_title('Our logbeta')
plt.show()

fig, ax = plt.subplots(figsize=(10, 3))
ax.pcolormesh(example["logbeta"].T, cmap='viridis')
ax.set_title('Correct logbeta')
plt.show()

# Viterbi



# Baum-welch


: 